# Caso práctico: Extractor de comandas

Una importante cadena de restaurantes quiere implantar un sistema de comandas por chat, de esta forma los clientes simplemente tendrán que escribir su comanda por una tablet en la mesa y el asistente virtual se encargará de gestionar el pedido con la cocina y los camareros.

El restaurante necesita tener un listado de los platos y las bebidas por separado junto con el número de unidades que se piden de cada elemento.

## Importación de librerias

Trabajaremos con la librería NLTK, con el tokenizador incluido en la librería, el corpus cess_esp y los 4 taggers vistos en la unidad. Tendremos que importar también el RegEx Parser de NLTK.

Por último, como complementos no obligatorio pero que utilizaremos para dar formato o hacer más sencillo el trabajo, train_test_split de sklearn y pandas.

In [1]:
#Importamos la librería base NLTK
import nltk

#Importamos el componente de NLTK para tokenizar
from nltk.tokenize import word_tokenize

#Importamos el corpus CESS en Español
from nltk.corpus import cess_esp

#Taggers ngrams y HMM
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger

#RegEx Parser
from nltk.chunk.regexp import *


#Esto nos permitirá crear los conjuntos de test y train
from sklearn.model_selection import train_test_split

#Importamos por último pands
import pandas as pd

Ahora ya tenemos todo lo necesario para emprezar a trabajar con los corpus y los taggers.

## Entrenamiento de los taggers

Para entrenar los taggers lo que tenemos que hacer es traernos el corpus taggeado en español. Antes, veamos que contiene ese corpus. 

In [2]:
cess_esp.sents()

[['El', 'grupo', 'estatal', 'Electricité_de_France', '-Fpa-', 'EDF', '-Fpt-', 'anunció', 'hoy', ',', 'jueves', ',', 'la', 'compra', 'del', '51_por_ciento', 'de', 'la', 'empresa', 'mexicana', 'Electricidad_Águila_de_Altamira', '-Fpa-', 'EAA', '-Fpt-', ',', 'creada', 'por', 'el', 'japonés', 'Mitsubishi_Corporation', 'para', 'poner_en_marcha', 'una', 'central', 'de', 'gas', 'de', '495', 'megavatios', '.'], ['Una', 'portavoz', 'de', 'EDF', 'explicó', 'a', 'EFE', 'que', 'el', 'proyecto', 'para', 'la', 'construcción', 'de', 'Altamira_2', ',', 'al', 'norte', 'de', 'Tampico', ',', 'prevé', 'la', 'utilización', 'de', 'gas', 'natural', 'como', 'combustible', 'principal', 'en', 'una', 'central', 'de', 'ciclo', 'combinado', 'que', 'debe', 'empezar', 'a', 'funcionar', 'en', 'mayo_del_2002', '.'], ...]

In [3]:
cess_esp.tagged_sents()

[[('El', 'da0ms0'), ('grupo', 'ncms000'), ('estatal', 'aq0cs0'), ('Electricité_de_France', 'np00000'), ('-Fpa-', 'Fpa'), ('EDF', 'np00000'), ('-Fpt-', 'Fpt'), ('anunció', 'vmis3s0'), ('hoy', 'rg'), (',', 'Fc'), ('jueves', 'W'), (',', 'Fc'), ('la', 'da0fs0'), ('compra', 'ncfs000'), ('del', 'spcms'), ('51_por_ciento', 'Zp'), ('de', 'sps00'), ('la', 'da0fs0'), ('empresa', 'ncfs000'), ('mexicana', 'aq0fs0'), ('Electricidad_Águila_de_Altamira', 'np00000'), ('-Fpa-', 'Fpa'), ('EAA', 'np00000'), ('-Fpt-', 'Fpt'), (',', 'Fc'), ('creada', 'aq0fsp'), ('por', 'sps00'), ('el', 'da0ms0'), ('japonés', 'aq0ms0'), ('Mitsubishi_Corporation', 'np00000'), ('para', 'sps00'), ('poner_en_marcha', 'vmn0000'), ('una', 'di0fs0'), ('central', 'ncfs000'), ('de', 'sps00'), ('gas', 'ncms000'), ('de', 'sps00'), ('495', 'Z'), ('megavatios', 'ncmp000'), ('.', 'Fp')], [('Una', 'di0fs0'), ('portavoz', 'nccs000'), ('de', 'sps00'), ('EDF', 'np00000'), ('explicó', 'vmis3s0'), ('a', 'sps00'), ('EFE', 'np00000'), ('que', 'c

Como vemos con el comando `cess_esp.sents()` nos traemos un conjunto de frases tokenizadas de distintas temáticas.

Y con el comando `cess_esp.tagged_sents()` nos traemos ese mismo conjunto de frases ya taggeadas.

Ahora lo que haremos será crear 2 conjuntos de tokens taggeados. Uno que contenga el 90% de los tokens y otro que contenga el 10%, uno para train y otro para test respectivamente.

In [4]:
#Generamos los conjuntos de Train y Test
data_train, data_test = train_test_split(cess_esp.tagged_sents(), test_size=0.10, random_state=1)

print('Token de entrenamiento:',len(data_train),
      '\nTokens de test:    ',len(data_test))

Token de entrenamiento: 5427 
Tokens de test:     603


Teniendo los conjuntos ya creados, pasamos a entrenar los taggers. 

Para entrenar los **ngram** deberemos ejecutar el tagger con el corpus, por ejemplo `UnigramTagger(data_train)`. Veremos que los ngram pueden tener como backoff otro ngram.

En el caso de **HiddenMarkovModelTagger** deberemos ejecutar la función `.train()`.



In [5]:
unigram  = UnigramTagger(data_train)
bigram   = BigramTagger(data_train, backoff=unigram)
trigram  = TrigramTagger(data_train, backoff=bigram)
hmm      = HiddenMarkovModelTagger.train(data_train)

Una vez entrenados los taggers, vamos a evaluar cómo es el tendimiento de cada uno de ellos con el conjunto de test. Para evaluarlo tenemos que utilizar la función `train()`, para todos los taggers. Veamos qué tal funciona cada uno de ellos.

Cuando ejecutes el entrenamiento presta atención al tiempo que tarda cada uno de los taggers en mostrar la puntuación. Mientras que los ngram son bastante rápidos para extraer la información, el HMM tarda más tiempo en obtener los datos.

In [6]:
print ('Acierto con unigramas: %.2f %%' % (unigram.evaluate(data_test)*100))
print ('Acierto con bigramas:  %.2f %%' % (bigram.evaluate(data_test)*100))
print ('Acierto con trigramas: %.2f %%' % (trigram.evaluate(data_test)*100))
print ('Acierto con HMMs:      %.2f %%' % (hmm.evaluate(data_test)*100))

Acierto con unigramas: 87.27 %
Acierto con bigramas:  88.78 %
Acierto con trigramas: 88.77 %
Acierto con HMMs:      89.57 %


Ahora, podemos volver a entrenar los taggers con los datos de test. Aunque no apreciaremos una gran mejora en terminos generales, como el volumen de datos que estamos utilizando es pequeño, nos será de ayuda.

Veremos, que si evaluamos los taggers de nuevo en el conjunto de test, obtendremos casi un 100% de acierto. Esta mejoría es mayor en los ngramas puesto que son reglas más 'logicas' por así decirlo, por lo que con pequeñs vólumenes de datos veremos mayor acierto. En cambio, en el HMM, vemos mejor mejoría en el caso de evaluar sobre los datos pero en aquellos tokens no entrenados tendremos mejor rendimiento que el resto

In [7]:
unigram  = UnigramTagger(data_test)
bigram   = BigramTagger(data_test, backoff=unigram)
trigram  = TrigramTagger(data_test, backoff=bigram)
hmm      = HiddenMarkovModelTagger.train(data_test)

print ('Acierto con unigramas: %.2f %%' % (unigram.evaluate(data_test)*100))
print ('Acierto con bigramas:  %.2f %%' % (bigram.evaluate(data_test)*100))
print ('Acierto con trigramas: %.2f %%' % (trigram.evaluate(data_test)*100))
print ('Acierto con HMMs:      %.2f %%' % (hmm.evaluate(data_test)*100))

Acierto con unigramas: 96.67 %
Acierto con bigramas:  98.86 %
Acierto con trigramas: 99.56 %
Acierto con HMMs:      93.62 %


Como hemos visto que el HMM tiene mejor rendimiento, a priori, utilizaremos este para elaborar nuestro bot de comandas.

## Comenzar a elaborar el bot

Ahora crearemos una frase de la temática de nuestro bot para ver cómo es el rendimiento con nuestro conjunto de datos.

In [8]:
food_text = 'Quiero unos macarrones con queso y una cerveza'

El primer paso será elaborar los tokens de la frase. Para los taggers no es necesario que eliminemos las stopwords, lematicemos o derivemos, al contrario. Si hacemos todos estos pasos estaremos eliminando información que utilizarán los taggers para encontrar las etiquetas.

In [9]:
tokens = nltk.word_tokenize(food_text)
tokens

['Quiero', 'unos', 'macarrones', 'con', 'queso', 'y', 'una', 'cerveza']

Una vez tengamos los tokens, utilzaremos la función `.tag()`

In [10]:
food_tagged = hmm.tag(tokens)
food_tagged

[('Quiero', 'da0mp0'),
 ('unos', 'di0mp0'),
 ('macarrones', 'ncmp000'),
 ('con', 'sps00'),
 ('queso', 'np0000l'),
 ('y', 'cc'),
 ('una', 'di0fs0'),
 ('cerveza', 'ncfs000')]

Podemos ver que los tags obtenidos no son del todo correctos. Para comprobarlos, debemos revisar los tags EAGLES del enlace facilitado en el temario: https://www.cs.upc.edu/~nlp/tools/parole-sp.html

De forma resumida, utilizaremos estos tags, aunque podemos tener variaciones e incorporar otros;

- **ncms000** : nombre común masculino singular
- **ncfs000** : nombre común femenino singular
- **ncmp000** : nombre común masculino plural
- **ncfp000** : nombre común femenino plural

- **np0000p/np00001** : nombre propio (La incluimos porque fijandonos en como etiqueta nuestros ejemplos podemos comprobar que para algunas palabras le pone este etiquetado(ej: bocadillo de atún). Tambien podría darse el caso de que alguna comida siguiera esta estructura correctamente (ej: pizza de Toni)


- **di0ms0** : determinante indefinido masculino singular
- **di0fs0** : determinante indefinido femenino singular
- **di0mp0** : determinante indefinido masculino plural
- **di0fp0** : determinante indefinido femenino plural
- **dn0cp0** : determinante indefinido comun plural


- **sps00** : Preposición


- **da0ms0**: el
- **da0fs0**: la
- **da0mp0**: los
- **da0fp0**: las
- **da0ns0**: lo

Repasemos la frase:

 * ('Quiero', 'da0mp0') -> Taggeado como un determinante, debería ser: vmpip1s0 que corresponde a presente de indicativo
 * ('unos', 'di0mp0') -> Taggeado como determinante masculino, debería ser: mcmp00 que corresponde a un numeral ordinal masculino
 * ('macarrones', 'ncmp000'), -> **Correcto**: Taggeado como Sustantivo Común Masculino Plural
 * ('con', 'sps00'), -> **Correcto**: Taggeado como preposición
 * ('queso', 'np0000l'), -> Taggeado como Sustantivo Propio, debería ser: ncms000 sustantivo común masculino singular
 * ('y', 'cc'),-> **Correcto**: Taggeado como conjunción coordinada
 * ('una', 'di0fs0'),-> Taggeado como determinante femenino, debería ser: mcfp00 que corresponde a un numeral ordinal femenino
 * ('cerveza', 'ncfs000')-> -> **Correcto**: Taggeado como sustantivo femenino singular
 
 Como hemos visto, correctamente taggeados tendríamos 4 tokens de 8, un 50% de aciertos. Si evaluamos estos tags con el rendimiento de, por ejemplo el trigram ¿Qué % de acierto tendremos? Comprobemoslo

In [11]:
print ('Acierto con unigramas: %.2f %%' % (unigram.evaluate([food_tagged])*100))
print ('Acierto con bigramas:  %.2f %%' % (bigram.evaluate([food_tagged])*100))
print ('Acierto con trigramas: %.2f %%' % (trigram.evaluate([food_tagged])*100))
print ('Acierto con HMMs:      %.2f %%' % (hmm.evaluate([food_tagged])*100))

Acierto con unigramas: 50.00 %
Acierto con bigramas:  50.00 %
Acierto con trigramas: 50.00 %
Acierto con HMMs:      100.00 %


Como vemos, HMM dice que su acierto es del 100%, tiene lógica puesto que es ese tagger el que ha hecho los tags. En cambio el resto de los tags, coinciden en la corrección que hemos hecho, aunque esto no quiere decir que esos tags acierten 8 de 8 etiquetas, comprobemoslo.

In [12]:
print ('Unigramas:', (unigram.tag(tokens)))
print ('Bigramas: ', (bigram.tag(tokens)))
print ('Trigramas: ', (trigram.tag(tokens)))

Unigramas: [('Quiero', None), ('unos', 'di0mp0'), ('macarrones', None), ('con', 'sps00'), ('queso', None), ('y', 'cc'), ('una', 'di0fs0'), ('cerveza', None)]
Bigramas:  [('Quiero', None), ('unos', 'di0mp0'), ('macarrones', None), ('con', 'sps00'), ('queso', None), ('y', 'cc'), ('una', 'di0fs0'), ('cerveza', None)]
Trigramas:  [('Quiero', None), ('unos', 'di0mp0'), ('macarrones', None), ('con', 'sps00'), ('queso', None), ('y', 'cc'), ('una', 'di0fs0'), ('cerveza', None)]


Ahora que hemos visto estos resultados, confirmamos que el que mejor rendimiento ha tenido es HMM, que ha encontrado etiquetas, que aunque no del todo correctas han sido aproximadas, en algunos casos ha confundido el género, en otros el número aunque esto no afectaría demasiado a nuestra extracción de información. Pero en general ha etiquetado en tipo de palabra 6 de los 8 tokens.

Los ngrams, no han podido identificar los tokens en el 50% de los casos. Por lo que habríamos tenido problemas a la hora de utilizarlos como taggers.

## Corregir el tagger

Ahora que hemos corregido los tags, es hora de volver a entrenar el tagger con la frase correcta, por lo que vamos a ello.
Vamos a también a trabajar con el foodTagger (es un HMM entrenado con nuestras frases de comida).

In [13]:
corrected_tokens = [('Quiero', 'vmpip1s0'), ('unos', 'mcmp00'), ('macarrones', 'ncmp000'), ('con', 'sps00'), ('queso', 'ncms000'), ('y', 'cc'), ('una', 'mcfp00'), ('cerveza', 'ncfs000')]

foodTagger = hmm.train([corrected_tokens])

Ahora, una vez entrenado el tagger, si volvemos a pasar la misma frase acertará con todos ellos puesto que está entrenado para detectar los tags.

In [14]:
food_tagged = foodTagger.tag(tokens)
food_tagged

[('Quiero', 'vmpip1s0'),
 ('unos', 'mcmp00'),
 ('macarrones', 'ncmp000'),
 ('con', 'sps00'),
 ('queso', 'ncms000'),
 ('y', 'cc'),
 ('una', 'mcfp00'),
 ('cerveza', 'ncfs000')]

La corrección del tagger es un proceso que deberemos elaborar repetidamente para conseguir mejorar los resultados. Es un proceso que podríamos acortar en gran manera si tuviesemos un corpus con miles de frases y tokens como el inicial, pero de nuestro contexto en concreto.

## Elaborar una función que reconozca las comandas

Para ello tendremos que utilizar RegEx Parser y las reglas lógicas. En nuestro caso, podemos utilizar las que hemos visto en la teoría de esta unidad.

- nombre común : *macarrones*

- nombre común + nombre (común/propio) : *pizza margarita*

- nombre común + preposición + nombre(común/propio) : *bocadillo de atún*

- nombre común + preposición + artículo + nombre(común/propio) : *lentejas a la riojana*

In [15]:
reglas = r'''
    cantidad: {<mccp00>}
    comida: {<ncms000|ncfs000|ncmp000|ncfp000>*<sps00>*<da0ms0|da0fs0|da0mp0|da0fp0|da0ns0>*<ncms000|ncfs000|ncmp000|ncfp000|np0000l|np0000p>}
    cantidad: {<di0ms0|di0fs0|di0mp0|di0fp0|dn0cp0|mcmp00|mcfp00> || <mcmp00>* || <mcfp00> }
      '''

Ahora ya tenemos la gramática creada, vamos a crear la función del regex que extraiga la información

In [16]:
RegexP = nltk.RegexpParser(reglas)

def parsear(phrase):
    return RegexP.parse(phrase)

In [17]:
frase_regex = parsear(corrected_tokens)
print(frase_regex)

(S
  Quiero/vmpip1s0
  (cantidad unos/mcmp00)
  (comida macarrones/ncmp000 con/sps00 queso/ncms000)
  y/cc
  (cantidad una/mcfp00)
  (comida cerveza/ncfs000))


### Función para extraer los nodos clasificados

Una vez hemos conseguido identificar la comida y la cantidad de la comanda, generaremos un JSON con los datos de la comanda

In [18]:
def genera_comanda(tree):
    
    result = []
    
    item = {}
    item['item'] = None
    item['cantidad'] = 0
    
    elementos = 0
    
    #En primer lugar contaremos cuantos elementos hay en el pedido
    for nodo in tree:
        if type(nodo) == tuple:
            continue
        tipo = nodo.label()
        if tipo == 'comida':
            elementos += 1
            
    #Ahora generaremos cada línea de pedido con sus cantidades
    for nodo in tree:
        if type(nodo) == tuple:
            continue
        
        count = 0
        valor = ''
        
        for elemento in nodo:
            count += 1
            palabra, categoria = elemento
                
            if count == 1:
                valor = valor + palabra
            else:
                valor = valor + ' ' + palabra
            
            if nodo.label() == 'cantidad':
                item['cantidad'] = valor
            else:
                item['item'] = valor
        
        if nodo.label() == 'comida':
            result.append(item)
            item = {}
            #print(item)
        
    
    return result

In [19]:
genera_comanda(frase_regex)

[{'item': 'macarrones con queso', 'cantidad': 'unos'},
 {'cantidad': 'una', 'item': 'cerveza'}]

Ahora que la función nos genera la comada, generaremos una nueva función que lo haga desde 0

In [20]:
def procesa_frase(frase):
    
    tokens = nltk.word_tokenize(frase)
    print('Tokens:')
    print(tokens)
    print('\n', '-----------------------------------------', '\n')
    tags = foodTagger.tag(tokens)
    print('TAGS:')
    print(tags)
    print('\n', '-----------------------------------------', '\n')
    parsed = parsear(tags)
    print('Parsed:')
    print(parsed)
    print('\n', '-----------------------------------------', '\n')
    
    return genera_comanda(parsed)    
    

Y por último, testeamos la función que hemos creado para analizar frases completas

In [21]:
fraseTest = 'pedir dos pizzas cuatro quesos y cinco fantas'

procesa_frase(fraseTest)

Tokens:
['pedir', 'dos', 'pizzas', 'cuatro', 'quesos', 'y', 'cinco', 'fantas']

 ----------------------------------------- 

TAGS:
[('pedir', 'vmpip1s0'), ('dos', 'mcmp00'), ('pizzas', 'ncmp000'), ('cuatro', 'sps00'), ('quesos', 'ncms000'), ('y', 'cc'), ('cinco', 'mcfp00'), ('fantas', 'ncfs000')]

 ----------------------------------------- 

Parsed:
(S
  pedir/vmpip1s0
  (cantidad dos/mcmp00)
  (comida pizzas/ncmp000 cuatro/sps00 quesos/ncms000)
  y/cc
  (cantidad cinco/mcfp00)
  (comida fantas/ncfs000))

 ----------------------------------------- 



[{'item': 'pizzas cuatro quesos', 'cantidad': 'dos'},
 {'cantidad': 'cinco', 'item': 'fantas'}]

Aunque esta frase funciona, es posible que otras no lo hagan, por ejemplo si no ponemos un verbo antes de la comanda, hagamos una prueba

In [22]:
fraseTest = 'dos pizzas cuatro quesos y cinco fantas'
procesa_frase(fraseTest)

Tokens:
['dos', 'pizzas', 'cuatro', 'quesos', 'y', 'cinco', 'fantas']

 ----------------------------------------- 

TAGS:
[('dos', 'vmpip1s0'), ('pizzas', 'mcmp00'), ('cuatro', 'ncmp000'), ('quesos', 'sps00'), ('y', 'ncms000'), ('cinco', 'cc'), ('fantas', 'mcfp00')]

 ----------------------------------------- 

Parsed:
(S
  dos/vmpip1s0
  (cantidad pizzas/mcmp00)
  (comida cuatro/ncmp000 quesos/sps00 y/ncms000)
  cinco/cc
  (cantidad fantas/mcfp00))

 ----------------------------------------- 



[{'item': 'cuatro quesos y', 'cantidad': 'pizzas'}]

En esta frase únicamente ha identificado, y de forma erronea, por lo que vamos a corregirla y a entrenar nuestro tagger

In [23]:
corrected_order = [('dos', 'mccp00'), ('pizzas', 'ncmp000'), ('cuatro', 'sps00'), ('quesos', 'ncms000'), ('y', 'cc'), ('cinco', 'mcfp00'), ('fantas', 'ncfs000')]

foodTagger = foodTagger.train([corrected_order])

Ya hemos entrenado de nuevo nuestro tagger, por lo que podemos volver a probar la misma frase y comprobar su rendimiento

In [24]:
procesa_frase(fraseTest)

Tokens:
['dos', 'pizzas', 'cuatro', 'quesos', 'y', 'cinco', 'fantas']

 ----------------------------------------- 

TAGS:
[('dos', 'mccp00'), ('pizzas', 'ncmp000'), ('cuatro', 'sps00'), ('quesos', 'ncms000'), ('y', 'cc'), ('cinco', 'mcfp00'), ('fantas', 'ncfs000')]

 ----------------------------------------- 

Parsed:
(S
  (cantidad dos/mccp00)
  (comida pizzas/ncmp000 cuatro/sps00 quesos/ncms000)
  y/cc
  (cantidad cinco/mcfp00)
  (comida fantas/ncfs000))

 ----------------------------------------- 



[{'item': 'pizzas cuatro quesos', 'cantidad': 'dos'},
 {'cantidad': 'cinco', 'item': 'fantas'}]

In [25]:
procesa_frase('dos hamburguesas')

Tokens:
['dos', 'hamburguesas']

 ----------------------------------------- 

TAGS:
[('dos', 'mccp00'), ('hamburguesas', 'ncmp000')]

 ----------------------------------------- 

Parsed:
(S (cantidad dos/mccp00) (comida hamburguesas/ncmp000))

 ----------------------------------------- 



[{'item': 'hamburguesas', 'cantidad': 'dos'}]

Y con esto ya tendríamos listo nuestro bot procesador de comandas. Por supuesto tiene margen de mejora y necesidad de mayor entrenamiento, pero tiene la funcionalidad requerida. 